## Sơ lược

Mô hình LightFM là một biến thể dựa trên thuật toán phân rã ma trận (matrix factorization collaborative - MF). điểm chung là đều ánh xạ user (item) về một vector biểu diễn bởi tập các feature. Tuy nhiên có vài điểm:
- MF coi tập feature cho mỗi user là các đánh giá của user đó, còn lightFM coi tập feature cho mỗi user là các meta data có sẵn, dẫn đến số lượng tham số cần tối ưu ít hơn hẳn
- Với một user mới, chưa có dữ liệu trước đó, thì lightFM xem vector biểu diễn cho user này là tổ hợp tuyến tính của các feature vector (học được từ trước đó), ví dụ như một user có giới tính Nam, là nghệ sĩ, độ tuổi > 45 thì lấy tổ hợp của các feature vector cho giới tính là Nam, cho nghề nghiệp là nghệ sĩ, dđộ tuổi > 45 để có biểu diễn cho user mới đó.
- Như vậy nếu không có meta data thì LightFM chính là MF thường
- Có thể thấy LightFM là một thuật toán kết hợp được tính chất của cả content-based (tự thân mỗi user/item biểu diễn bởi một vector) và collaborative filtering (tri thức chuyển giao từ các neighbor khác, thông qua meta data)

Mô hình: Đặt $U$ là tập user, $I$ là tập item, $F^U$ là tập các feature cho user, $F^I$ là tập các feature cho item. Ở vị trí rating $\neq 0$ ta coi là có tương tác giữa user-item, tập này ta kí hiệu  $(u,i)\in S^+$ (positive interaction), đánh dấu là 1, ngược lại đánh dấu 0, kí hiệu $(u,i) \in S^-$. \\
Với mỗi user $u$ biễu diễn bởi tập các features $f_u ⊂ F^U$. Tương tự cho item $i$ biễu diễn bởi features $f_i ⊂ F^I$. \\
Tham số của mô hình là các vector embedding d-chiều (latent vectors) $e^U_f$ và $e^I_f$ cho mỗi feature f \\
Biễu diễn của một user u (liên hệ đến ví dụ về user là nghệ sĩ ở trên) chính là tổng
$$q_u = \sum_{j \in f_u} e^U_j$$
Với item thì
$$p_i = \sum_{j \in f_i} e^I_j$$
Các tham số bias:
$$b_u = \sum_{j\in f_u}b^U_j$$
$$b_i = \sum_{j\in f_i}b^I_j$$
dự đoán rating của user i to item j là
$$\hat{r}_{ui} = f(q_u\cdot p_i+b_u+b_i)$$
trong đó f là hàm sigmoid $f(x) = \frac{1}{1+e^{-x}}$ \\
Hàm mục tiêu là đi tối ưu (maximimum likelihood)
$$L(e^U, e^I, b^U, b^I) = \prod_{(u,i)\in S^+}\hat{r}_{ui} \times \prod_{(u,i)\in S^-} (1-\hat{r}_{ui}) $$
tương đương với việc tìm cực tiểu hàm negative log-likelihood của

$\Rightarrow$ Một số hàm loss thường được ưa chuộng là BPR (Bayesian personalized ranking) và WARP (Weighted approxiamte rank pairwise)
- Với BPR Loss, hàm này khá hữu ích khi nó xem xét thêm vấn đề cặp dữ liệu (pairwise), ví dụ như user u ưa thích item i hơn item j, kí hiệu sự kiện này là $>_u$ thì ta có thể hình thức hóa nó dưới dạng xác suất có điều kiện $P(\theta|>_u) \propto P(>_u|\theta)P(\theta)$ và đi cực đại hóa đại lượng này
- Hoặc với WARP Loss, xem xét một user u và $f_u(i)$ là các giá trị thể hiện đánh giá cho các item i (có thể = 0), $D_u$ là tập các đánh giá mà user u đã thực hiện, mục tiêu là đi cực tiểu hóa đại lượng $$L = \sum_{i\in D_u}L(rank_i(f_u))$$
trong đó $rank_i(f_u) = \sum_{j \notin D_u} I\{f_u(j) \geq f_u(i)\}$, $I(\cdot)$ là hàm chỉ báo (1 hoặc 0), $L(\cdot)$ là một loại phép biến đổi chuyển từ rời rạc sang liên tục (để có thể lấy gradient được). Cụ thể:
$$L ( rank_i(f_u) )
= \sum_{j \notin D_u} log ( rank_i(f_u) ) \frac{| 1 - f_u(i) + f_u(j) |_+}{rank_i(f(u))}$$


## Chuẩn bị dữ liệu

Tải thư viện lightFM và các thư viện cần thiết

In [ ]:
pip install lightfm

In [ ]:
import warningswarnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import copy
import tqdm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import numpy as np
from lightfm.cross_validation import random_train_test_split
import os
from scipy.sparse import csr_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Đường dãn

In [ ]:
path = '/content/drive/MyDrive/ml-1m/ml-1m/'

### Dữ liệu ratings

Đoc Dữ liệu rating

In [ ]:
ratings_df = pd.read_csv(f"{path}ratings.dat", delimiter = "::", encoding="latin-1", header = None, engine = 'python',
                      names = ['user_id', 'movie_id', 'rating', 'time']
)
ratings_df

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


### Dữ liệu movies

In [ ]:
movies_df = pd.read_csv(f"{path}movies.dat", delimiter = "::", encoding="latin-1", header = None, engine = 'python',
                     names = ['movie_id', 'movie_name', 'genre'])

In [ ]:
movies_df

,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


### Dữ liệu users

In [ ]:
readme_text = np.array(open('/content/drive/MyDrive/ml-1m/ml-1m/README').read().splitlines())
start_index = np.flatnonzero(np.core.defchararray.find(readme_text,'Occupation is chosen')!=-1)[0]
end_index = np.flatnonzero(np.core.defchararray.find(readme_text,'MOVIES FILE DESCRIPTION')!=-1)[0]
occupation_list = [x.split('"')[1] for x in readme_text[start_index:end_index][2:-1].tolist()]
occupation_dict = dict(zip(range(len(occupation_list)), occupation_list))
users_df = pd.read_csv(f'{path}users.dat', delimiter = '::', engine = 'python',
                    header = None, names = ['user_id', 'gender', 'age', 'occupation', 'zip_code'])
users_df['occupation'] = users_df['occupation'].replace(occupation_dict)
users_df

,user_id,gender,age,occupation,zip_code
0,1,F,1,K-12 student,48067
1,2,M,56,self-employed,70072
2,3,M,25,scientist,55117
3,4,M,45,executive/managerial,02460
4,5,M,25,writer,55455
...,...,...,...,...,...
6035,6036,F,25,scientist,32603
6036,6037,F,45,academic/educator,76006
6037,6038,F,56,academic/educator,14706
6038,6039,F,45,other,01060


### Trích xuất vector Tfidf

In [ ]:
def extract_info(string):
    x = re.split(" \((\d{4})\)$",string)
    return x[:2]

In [ ]:
def genre_extract(string):
    return re.split("\|",string)

In [ ]:
def all_genre_creating(movies):
    uniqe_genres = []
    for string in movies['genre']:
        array = genre_extract(string)
        for genre in array:
            if genre not in uniqe_genres and genre != '(no genres listed)':
                uniqe_genres.append(genre)
    return uniqe_genres

In [ ]:
def movie_profiling(movies):
    uniqe_genre = all_genre_creating(movies)
    movie_profile = copy.deepcopy(movies)
    for genre in uniqe_genre :
        movie_profile[genre] = 0
    loc = 0
    for string in movie_profile['genre']:
        for genre in genre_extract(string):
            if genre in uniqe_genre:
                movie_profile[genre].iloc[loc] = 1
        loc = loc + 1

    movie_profile = movie_profile.drop(columns=['movie_name', 'genre']).set_index('movie_id')
    return movie_profile

In [ ]:
movie_profile = movie_profiling(movies_df)
movie_profile

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
movie_id,,,,,,,,,,,,,,,,,,
1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3949,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3950,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
ratings = ratings_df.drop(columns = ['timestamp'])

In [ ]:
user_x_movie = pd.pivot_table(ratings_df, values='rating', index=['movie_id'], columns = ['user_id'])
user_x_movie.sort_index(axis=0, inplace=True)
userIDs = user_x_movie.columns
user_profile = pd.DataFrame(columns = movie_profile.columns)

for i in range(len(user_x_movie.columns)):
  working_df = movie_profile.mul(user_x_movie.iloc[:,i], axis=0)
  user_profile.loc[userIDs[i]] = working_df.mean(axis=0)

In [ ]:
df = movie_profile.sum()
idf = (len(movies_df)/df).apply(np.log)
TFIDF = movie_profile.mul(idf.values)
# recommendation prediction
# df_predict = pd.DataFrame()

# for i in range(len(user_x_movie.columns)):
#   working_df = TFIDF.mul(user_profile.iloc[i], axis=1)
#   df_predict[user_x_movie.columns[i]] = working_df.sum(axis=1)

In [ ]:
TFIDF

### Biễu diễn một item qua các feature (lấy từ trích xuất tfidf)

In [ ]:
TFIDF.reset_index(inplace=True)
TFIDF.rename(columns={'movie_id': 'movie_id'}, inplace=True)
TFIDF

,movie_id,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,3.610403,2.73891,1.174286,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.000000,2.73891,0.000000,2.618916,4.044856,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.000000,0.00000,1.174286,0.000000,0.000000,2.109505,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.000000,0.00000,1.174286,0.000000,0.000000,0.000000,0.884731,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.000000,0.00000,1.174286,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,0.000000,0.00000,1.174286,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,3949,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.884731,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,3950,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.884731,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,3951,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.884731,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Biễu diễn một user qua các feature (lấy từ meta data)

Ta có thể tạo một feature mới là khoảng độ tuổi để giảm bớt số chiều của vector biểu diễn chp user  

In [ ]:
users_df['age_bin'] = pd.cut(users_df['age'], bins=[0,25,30,45,np.inf], labels= ['<= 25', '26 - 30', '31 - 45', '>= 45'])
users_df.head()

,user_id,gender,age,occupation,zip_code,age_bin
0,1,F,1,K-12 student,48067,<= 25
1,2,M,56,self-employed,70072,>= 45
2,3,M,25,scientist,55117,<= 25
3,4,M,45,executive/managerial,02460,31 - 45
4,5,M,25,writer,55455,<= 25


Bỏ bót feature 'age', 'zip-code', và biễu diễn feature dưới dạng one-hot encoding

In [ ]:
user = pd.get_dummies(users_df.drop(columns = ['age', 'zip_code'])) # for simple
user_features_col = user.drop(columns =['user_id']).columns.values
user_feat = user.drop(columns =['user_id']).to_dict(orient='records')
user.head()

,user_id,gender_F,gender_M,occupation_K-12 student,occupation_academic/educator,occupation_artist,occupation_clerical/admin,occupation_college/grad student,occupation_customer service,occupation_doctor/health care,...,occupation_scientist,occupation_self-employed,occupation_technician/engineer,occupation_tradesman/craftsman,occupation_unemployed,occupation_writer,age_bin_<= 25,age_bin_26 - 30,age_bin_31 - 45,age_bin_>= 45
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,3,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [ ]:
item_features = movies_df
item_features_col = TFIDF.drop(columns = ['movie_id']).columns.values
item_feat = TFIDF.drop(columns = ['movie_id']).to_dict(orient= 'records')

## Mô hình

Fit dữ liệu bằng method Dataset của LightFM, LightFM sẽ đọc bằng index của users_df và movies_df, tránh bị lệch index, sau dó khởi tạo ma trận user-item interaction

In [ ]:
dataset = Dataset()
dataset.fit(users=[x for x in user['user_id']], items=[x for x in TFIDF['movie_id']], item_features=item_features_col, user_features=user_features_col)
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 6040, num_items 3883.


Xây dựng item_features

In [ ]:
item_features = dataset.build_item_features((x,y) for x,y in zip(item_features['movie_id'],item_feat))

Xây dựng user_features

In [ ]:
user_features = dataset.build_user_features((x,y) for x,y in zip(user['user_id'],user_feat))

Xây dựng ma trận interaction

In [ ]:
(interactions, weights) = dataset.build_interactions((x, y)
                                                      for x,y in zip(ratings_df['user_id'], ratings_df['movie_id']))

In [ ]:
interactions

<6040x3883 sparse matrix of type '<class 'numpy.int32'>'
	with 1000209 stored elements in COOrdinate format>

ma trận user-item (interactions) có kích thước 6040 x 3883 ($\approx 22 \times 10^6$) và chỉ có $10^6$ dữ liệu có sẵn ($\approx 4.5$%)

+ Có một điều đáng lưu ý: weights ở đây là gì?
+ Ví dụ user i cho item j rating là 4 thì tương ứng interaction giữa chúng là 1, weight là 4. Mục đích là khi tính Loss giữa $\hat{r}_{ui}$ và $r_{ui}$, ta nhân thêm weight = 4 để ép cho magnitude của gradient là lớn, tham số khi update sẽ có bước nhảy lớn hơn, **hội tụ nhanh**, **chính xác** hơn.
+ Câu hỏi đặt ra: Nhanh có vẻ đúng, nhưng liệu có chính xác, hay cụ thể là vấn đề tham số có hội tụ về minima không, hay chỉ nhảy loanh quanh đó (do bước nhảy lớn)?

Theo docs, LightFM sử dụng cơ chế điều chỉnh learning rate bằng thuật toán AdaGrad, phương trình như sau:
$$\theta_t:=\theta_t - \frac{\eta}{\sqrt{s_t+ϵ}}\cdot g_t $$
với $s_t = s_{t-1} + g_t^2$

Khác với gradient descent thông thường: $\theta_t:=\theta_t - \eta \cdot g_t$

Thì AdaGrad update mỗi chiều của tham số theo cách khác nhau, tùy vào mẫu số, nếu mẫu số lớn, tức là chiều đó của tham số đang được update quá nhiều lần, AdaGrad sẽ điều chỉnh độ lớn gradient cho phù hợp, để nhường các lần update tiếp theo cho các chiều khác của tham số.

In [ ]:
# train test
train, test = random_train_test_split(interactions,test_percentage=0.1, random_state=779)
train_w, test_w = random_train_test_split(weights, test_percentage=0.1, random_state=779)

Ta dùng bộ tham số

In [ ]:
# parameter
n_components = 32 # số latent factors
loss = 'warp'
epoch = 100
num_thread = 4 # số physical cores của cpu
model = LightFM(no_components= n_components, loss=loss, random_state = 1616)
model.fit(train,  user_features= user_features, item_features= item_features, epochs=epoch,num_threads = num_thread, sample_weight = train_w)

- LightFM cung cấp sẵn vài metric, ví dụ như precison@k là tỉ lệ mà top k item đề xuất bởi mô hình đã xuất hiện trong dữ liệu có sẵn

- AUC score thể hiện rằng:  xác suất khi chọn một item bất kỳ (có interaction là 1, positive example) thì nó thực sự được đánh giá cao hơn item khác (có interaction là 0, negative example)

- Thường ta quan tâm đến AUC score hơn, vì nó phản ánh chất lượng toàn thể (overall performance) của mô hình

In [ ]:
train_precision = precision_at_k(model, train, k=10,item_features=item_features, user_features=user_features).mean()
test_precision = precision_at_k(model, test,train_interactions=train, k=10,item_features=item_features, user_features=user_features).mean()

# train_recall = recall_at_k(model, train, k=10,item_features=item_features, user_features=user_features).mean()
# test_recall = recall_at_k(model, test,train_interactions=train, k=10,item_features=item_features, user_features=user_features).mean()

train_auc = auc_score(model, train,item_features=item_features, user_features=user_features).mean()
test_auc = auc_score(model, test, train_interactions=train,item_features=item_features, user_features=user_features).mean()

print('Precision: train %.2f' % (train_precision))
print('Precision: test %.2f' % (test_precision))

# print('Recall: train %.2f' % (train_recall))
# print('Recall: test %.2f' % (test_recall))

print('AUC: train %.2f' % (train_auc))
print('AUC: test %.2f' % (test_auc))

Precision: train 0.59
Precision: test 0.19
AUC: train 0.93
AUC: test 0.92


##Gợi ý

### gợi ý cho user có sẵn

Chọn user có sẵn, chẳng hạn user có user_id = 4, đưa vào model, kết quả ra các item được đánh giá cao

In [ ]:
scores = model.predict(3, np.arange(100))
top_items = movies_df.iloc[np.argsort(-scores)]

In [ ]:
top_items.index

Int64Index([290, 606, 912, 257, 537, 161, 907,  69, 349,   5,
            ...
            488, 967, 619, 669,  55, 805,  50, 281, 554, 283],
           dtype='int64', length=1000)

user này có giới tính nam, nghề nghiệp là quản lí, độ tuổi 31-45

In [ ]:
user.iloc[3]

user_id                            4
gender_F                           0
gender_M                           1
occupation_K-12 student            0
occupation_academic/educator       0
occupation_artist                  0
occupation_clerical/admin          0
occupation_college/grad student    0
occupation_customer service        0
occupation_doctor/health care      0
occupation_executive/managerial    1
occupation_farmer                  0
occupation_homemaker               0
occupation_lawyer                  0
occupation_other                   0
occupation_programmer              0
occupation_retired                 0
occupation_sales/marketing         0
occupation_scientist               0
occupation_self-employed           0
occupation_technician/engineer     0
occupation_tradesman/craftsman     0
occupation_unemployed              0
occupation_writer                  0
age_bin_<= 25                      0
age_bin_26 - 30                    0
age_bin_31 - 45                    1
a

top 50 item ưa thích của user này là

In [ ]:
top_items[0:50][['movie_name','movie_id','genre']] #

,movie_name,movie_id,genre
69,From Dusk Till Dawn (1996),70,Action|Comedy|Crime|Horror|Thriller
5,Heat (1995),6,Action|Crime|Thriller
91,Vampire in Brooklyn (1995),93,Comedy|Romance
24,Leaving Las Vegas (1995),25,Drama|Romance
9,GoldenEye (1995),10,Action|Adventure|Thriller
93,Broken Arrow (1996),95,Action|Thriller
1,Jumanji (1995),2,Adventure|Children's|Fantasy
80,Things to Do in Denver when You're Dead (1995),81,Crime|Drama|Romance
0,Toy Story (1995),1,Animation|Children's|Comedy
41,Dead Presidents (1995),42,Action|Crime|Drama


Khi đối chiếu với những item mà user này đã đánh giá (dữ liệu quá khứ) ta thấy có ít hoặc không có sự trùng lặp  \\
Tức ở khía cạnh nào đó mô hình sẽ cho ra những gợi ý mới, phong phú hơn.

In [ ]:
known_rating = ratings_df[(ratings_df['user_id']==users_df['user_id'][3])][['movie_id','rating']].merge(movies_df[['movie_id','movie_name']], on = 'movie_id')
known_rating[known_rating['movie_id'].isin(top_items['movie_id'][0:50])] # => không có

,movie_id,rating,movie_name


### tìm các items tương tự nhau

movie có index 100 là 1 phim hài

In [ ]:
movies_df.iloc[100]

movie_id                   102
movie_name    Mr. Wrong (1996)
genre                   Comedy
Name: 100, dtype: object

ta query ra 10 phim tương tự

In [ ]:
def similar_items(item_id, model, N=10, norm = True):
    item_bias ,item_representations = model.get_item_representations(features=item_features)

    # Cosine similarity
    scores = item_representations.dot(item_representations[item_id, :])
    item_norms = np.linalg.norm(item_representations, axis=1)

    if norm == True:
        scores /= item_norms
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best]/ item_norms[item_id] ), key=lambda x: -x[1])
    else:
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best] ), key=lambda x: -x[1])
    return similar
similar_item_list = similar_items(100, model)
similar_idx = [x[0] for x in similar_item_list ]
movies_df.iloc[similar_idx][['movie_name', 'genre']]


,movie_name,genre
100,Mr. Wrong (1996),Comedy
272,Mixed Nuts (1994),Comedy
501,North (1994),Comedy
1151,Dear God (1996),Comedy
245,Houseguest (1994),Comedy
210,Bushwhacked (1995),Comedy
514,"Road to Wellville, The (1994)",Comedy
650,Eddie (1996),Comedy
629,Theodore Rex (1995),Comedy
1786,Krippendorf's Tribe (1998),Comedy


$\Rightarrow$ mô hình cho ra những kết quả rất tương tự nhau, chứng tỏ vector embeddings học được biểu diễn khá tốt cho các item

### Gợi ý cho user mới (cold-start problem)

Ta đưa 1 user bất kỳ vào, giả sử giới tính là Nam, nghề nghiệp nghệ sĩ, khoảng tuổi >= 45 và recommend 10 item cho user này

In [ ]:
new_user = pd.DataFrame(np.zeros(len(user_features_col))).T
new_user.columns = user_features_col
new_user['gender_M'] = 1
new_user['occupation_artist'] = 1
new_user['age_bin_>= 45'] = 1
new_user = csr_matrix(new_user)
scores_new_user = model.predict(user_ids = 0,item_ids = np.arange(interactions.shape[1]), user_features=new_user)
top_items_new_user = movies_df.iloc[np.argsort(-scores_new_user)]
top_items_new_user[0:10][['movie_id','movie_name', 'genre']]

,movie_id,movie_name,genre
244,247,Heavenly Creatures (1994),Drama|Fantasy|Romance|Thriller
2623,2692,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
1159,1175,Delicatessen (1991),Comedy|Sci-Fi
3682,3751,Chicken Run (2000),Animation|Children's|Comedy
1847,1916,Buffalo 66 (1998),Action|Comedy|Drama
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1244,1264,Diva (1981),Action|Drama|Mystery|Romance|Thriller
957,969,"African Queen, The (1951)",Action|Adventure|Romance|War
907,919,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
2843,2912,"Limey, The (1999)",Action|Crime|Drama
